DL MODEL

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip

In [ ]:
!unzip glove.6B.zip

In [2]:
%ls

 Volume in drive C is OS
 Volume Serial Number is 409B-89BD

 Directory of C:\Users\ry981

15-07-2024  11:12    <DIR>          .
27-04-2024  20:13    <DIR>          ..
02-07-2024  16:35    <DIR>          .anaconda
21-06-2024  10:36    <DIR>          .cache
15-07-2024  17:49    <DIR>          .conda
14-06-2024  09:09                25 .condarc
14-06-2024  09:08    <DIR>          .continuum
07-06-2024  19:17               179 .gitconfig
14-07-2024  20:50    <DIR>          .ipynb_checkpoints
14-06-2024  09:11    <DIR>          .ipython
14-06-2024  20:39    <DIR>          .jupyter
21-06-2024  10:10    <DIR>          .keras
15-06-2024  15:09    <DIR>          .matplotlib
21-06-2024  10:06    <DIR>          .virtual_documents
20-11-2023  14:25    <DIR>          .vscode
16-02-2024  18:28             7,145 -1.14-windows.xml
18-11-2023  18:20    <DIR>          ansel
25-02-2024  21:15            67,077 battery-report.html
18-11-2023  17:00    <DIR>          Contacts
18-11-2023  17:02    <DIR>   

In [4]:
from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file=r"C:\Users\ry981\glove.6B.200d.txt"
word2vec_output_file=r"D:\word2vec.txt.txt"
glove2word2vec(glove_input_file,word2vec_output_file)

C:\Users\ry981\AppData\Local\Temp\ipykernel_29536\2477500379.py:4: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_input_file,word2vec_output_file)


(400000, 200)

In [6]:
from gensim.models import KeyedVectors
model= KeyedVectors.load_word2vec_format(word2vec_output_file)

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential

# Load the dataset
data = pd.read_csv(r"D:\dataset.csv")

# Preprocess the data
texts = data['Text'].values
labels = data['Label'].values

# Encode labels
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Tokenize the text
tokenizer = Tokenizer(num_words=15000)
tokenizer.fit_on_texts(texts)
X = tokenizer.texts_to_sequences(texts)
X = pad_sequences(X, maxlen=200)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [10]:
def glove_embeddings(filepath, word_index, embedding_dim):
    embeddings_index = {}
    with open(filepath, 'r', encoding='utf8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs

    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

embedding_dim = 200
glove_filepath =r"C:\Users\ry981\glove.6B.200d.txt"
embedding_matrix = glove_embeddings(glove_filepath, tokenizer.word_index, embedding_dim)

In [12]:
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout, Activation
# Build the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=embedding_dim, weights=[embedding_matrix], input_length=200))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.3))
model.add(Dense(256,activation='relu'))

model.add(Dropout(0.4))
model.add(Dense(256,activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(32,activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64,callbacks=[early_stopping])

Epoch 1/10


D:\Anaconda\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


220/220 ━━━━━━━━━━━━━━━━━━━━ 197s 850ms/step - accuracy: 0.5942 - loss: 0.6753
Epoch 2/10
220/220 ━━━━━━━━━━━━━━━━━━━━ 187s 852ms/step - accuracy: 0.7300 - loss: 0.5235
Epoch 3/10
220/220 ━━━━━━━━━━━━━━━━━━━━ 206s 937ms/step - accuracy: 0.7894 - loss: 0.4280
Epoch 4/10
220/220 ━━━━━━━━━━━━━━━━━━━━ 211s 960ms/step - accuracy: 0.8439 - loss: 0.3405
Epoch 5/10
220/220 ━━━━━━━━━━━━━━━━━━━━ 210s 952ms/step - accuracy: 0.8957 - loss: 0.2458
Epoch 6/10
220/220 ━━━━━━━━━━━━━━━━━━━━ 213s 966ms/step - accuracy: 0.9331 - loss: 0.1781
Epoch 7/10
220/220 ━━━━━━━━━━━━━━━━━━━━ 214s 971ms/step - accuracy: 0.9488 - loss: 0.1356
Epoch 8/10
220/220 ━━━━━━━━━━━━━━━━━━━━ 218s 989ms/step - accuracy: 0.9663 - loss: 0.0992
Epoch 9/10
220/220 ━━━━━━━━━━━━━━━━━━━━ 224s 1s/step - accuracy: 0.9713 - loss: 0.0796
Epoch 10/10
220/220 ━━━━━━━━━━━━━━━━━━━━ 222s 1s/step - accuracy: 0.9790 - loss: 0.0665


In [14]:
from sklearn.metrics import classification_report
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype("int32")
y_pred = y_pred.flatten()
print(classification_report(y_test,y_pred))

110/110 ━━━━━━━━━━━━━━━━━━━━ 11s 95ms/step
              precision    recall  f1-score   support

           0       0.74      0.73      0.74      2094
           1       0.61      0.62      0.62      1426

    accuracy                           0.69      3520
   macro avg       0.68      0.68      0.68      3520
weighted avg       0.69      0.69      0.69      3520



In [16]:
model.export('best_model.pth')

INFO:tensorflow:Assets written to: best_model.pth\assets


INFO:tensorflow:Assets written to: best_model.pth\assets


Saved artifact at 'best_model.pth'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 200), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  1854321968048: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321977200: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321977552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321978080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321978256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321978432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321980016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321980368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321979664: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321980720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1854321980896: TensorSpec(shape=(), dtype=tf